# Testing `ipcmagic` with MPI4Py

Points to look at:
 * `%ipcluster start -n 12 --mpi` starts the cluster. On the terminal the following command should look like this:
    ```bash
    > ps -u $USER | grep ip
    1270 pts/8    00:00:01 ipcontroller
    1278 pts/8    00:00:00 ipcontroller
    1279 pts/8    00:00:00 ipcontroller
    1282 pts/8    00:00:00 ipcontroller
    1285 pts/8    00:00:00 ipcontroller
    1334 ?        00:00:01 ipengine
    1335 ?        00:00:01 ipengine
    1336 ?        00:00:00 ipengine
    1337 ?        00:00:00 ipengine
    1338 ?        00:00:00 ipengine
    1339 ?        00:00:00 ipengine
    1340 ?        00:00:00 ipengine
    1341 ?        00:00:00 ipengine
    1342 ?        00:00:00 ipengine
    1343 ?        00:00:00 ipengine
    1344 ?        00:00:00 ipengine
    1345 ?        00:00:00 ipengine
    31037 ?       00:00:00 slurm_script
    ```
 * `%ipcluster stop` stops the cluster. Check with the same command that all the process shown above no longer exist.
 * After stopping the cluster, it can be restarted by running `%ipcluster start -n 12 --mpi` again.
 * Running `%ipcluster start <...>` while the cluster is active, should give `IPCluster is already running`.
 * Running `%ipcluster stop` whould give `IPCluster not running`.
 * Restarting the kernel kills all the process mentioned above.

In [1]:
import numpy as np
import ipcmagic
import ipyparallel as ipp

In [2]:
%ipcluster --help

engine an IPyParallel cluster.

Usage:
  %ipcluster start -n <num_engines> [options]
  %ipcluster stop
  %ipcluster (-h | --help)
  %ipcluster --version

Options:
  -h --help                Show this screen.
  -v --version             Show version.
  -n --num_engines <int>   Number of engines (default 1 per node).
  -m --mpi                 Run with mpi support


In [3]:
%ipcluster --version

0.0.1


In [4]:
%ipcluster start -n 12 --mpi

IPCluster is ready! (7 seconds)


In [5]:
c = ipp.Client()   # (profile='job_17669451')

In [6]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [7]:
%%px
import socket
from mpi4py import MPI

In [8]:
%%px
print("Hello World from rank %s of %s on nid %s" % (MPI.COMM_WORLD.rank,
                                                    MPI.COMM_WORLD.size,
                                                    socket.gethostname()))

[stdout:0] Hello World from rank 10 of 12 on nid nid02891
[stdout:1] Hello World from rank 4 of 12 on nid nid02891
[stdout:2] Hello World from rank 7 of 12 on nid nid02891
[stdout:3] Hello World from rank 11 of 12 on nid nid02891
[stdout:4] Hello World from rank 9 of 12 on nid nid02891
[stdout:5] Hello World from rank 5 of 12 on nid nid02891
[stdout:6] Hello World from rank 3 of 12 on nid nid02891
[stdout:7] Hello World from rank 6 of 12 on nid nid02891
[stdout:8] Hello World from rank 2 of 12 on nid nid02891
[stdout:9] Hello World from rank 1 of 12 on nid nid02891
[stdout:10] Hello World from rank 0 of 12 on nid nid02891
[stdout:11] Hello World from rank 8 of 12 on nid nid02891


In [9]:
view = c[:]
view.activate() # enable magics

In [10]:
# run the contents of the file on each engine:
view.run('psum.py')

FileNotFoundError: [Errno 2] No such file or directory: 'psum.py'

In [ ]:
view.scatter('a',np.arange(16,dtype='float'))

In [ ]:
view['a']

In [ ]:
%px totalsum = psum(a)

In [ ]:
view['totalsum']

In [ ]:
%ipcluster stop